In [11]:
import csv
from scapy.all import *

In [4]:
packet_list = []
packet_list.append(['Src_MAC', 'Dest_MAC', 'Time', 'Len'])

In [7]:
def process_packet(packet):
    packet1 = []
    packet1.append(packet.src)
    packet1.append(packet.dst)
    packet1.append(packet.time)
    packet1.append(packet.len)
    packet_list.append(packet1)
#     print("SRC MAC: ", packet.src)
#     print("DST MAC: ", packet.dst)
#     print("ARRIVAL_TIME : ", packet.time)
#     print("LEN : ", packet.len)

In [9]:
sniff(offline="sample.pcap", prn=process_packet, store=0)
packet_list

[['Src_MAC', 'Dest_MAC', 'Time', 'Len'],
 ['60:32:b1:64:50:6d', 'ff:ff:ff:ff:ff:ff', Decimal('1646957193.773101'), 354],
 ['60:32:b1:64:50:6d', '01:00:5e:00:00:fb', Decimal('1646957198.281605'), 72],
 ['60:32:b1:64:50:6d', '01:00:5e:00:00:fb', Decimal('1646957198.292059'), 72],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.127740'), 408],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.127752'), 417],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.128224'), 480],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.128230'), 476],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.128628'), 456],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129194'), 488],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129200'), 470],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129788'), 472],
 ['60:32:b1:64:50:6d', '01:00:5e:7f:ff:fa', Decimal('1646957199.129794'), 472],
 

In [12]:
with open('740_csv.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(packet_list)

In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier # Import Random Forest Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, confusion_matrix
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import plot_roc_curve
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt  
import flow_stats
import time

In [49]:
filename = "sample2.csv"
dfa = pd.read_csv(filename)
dfa

,Unnamed: 0,source_port,dest_port,source_ip,dest_ip,source_mac,dest_mac,flow_duration,flow_rate,flow_volume,domain
0,t0,60656,443,192.168.0.100,54.212.65.189,ee:3a:28:eb:91:a0,60:32:b1:64:50:6d,888.226188,0.752061,77168,NaN
1,t1,58416,443,192.168.0.176,3.141.212.7,ee:3a:28:eb:91:a0,60:32:b1:64:50:6d,889.955122,0.569692,67092,NaN
2,t2,443,58388,3.138.39.171,192.168.0.176,ee:3a:28:eb:91:a0,28:6d:97:77:cf:e8,0.510249,74.473458,10718,NaN
3,t3,443,40176,52.205.2.10,192.168.0.176,ee:3a:28:eb:91:a0,28:6d:97:77:cf:e8,0.430702,88.228062,10718,NaN
4,t4,34772,443,192.168.0.100,52.88.211.192,ee:3a:28:eb:91:a0,60:32:b1:64:50:6d,0.727961,63.190193,14416,NaN
...,...,...,...,...,...,...,...,...,...,...,...
119,u95,34592,5353,192.168.0.1,224.0.0.251,60:32:b1:64:50:6d,01:00:5e:00:00:fb,0.000000,1.000000,86,192.168.0.188.in-addr.arpa
120,u96,42915,5353,192.168.0.1,224.0.0.251,60:32:b1:64:50:6d,01:00:5e:00:00:fb,0.000000,1.000000,86,192.168.0.188.in-addr.arpa
121,u97,41132,5353,192.168.0.1,224.0.0.251,60:32:b1:64:50:6d,01:00:5e:00:00:fb,0.000000,1.000000,86,192.168.0.113.in-addr.arpa
122,u98,49372,5353,192.168.0.1,224.0.0.251,60:32:b1:64:50:6d,01:00:5e:00:00:fb,0.000000,1.000000,86,192.168.0.113.in-addr.arpa


In [62]:
df = dfa.copy()
data = df.iloc[:,[1,2,3,4,6,7,8,9]]
labels = df.iloc[:,[5]]
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size=0.4, random_state=1 )
x_test

,source_port,dest_port,source_ip,dest_ip,dest_mac,flow_duration,flow_rate,flow_volume
48,40917,5353,192.168.0.1,224.0.0.251,01:00:5e:00:00:fb,0.000000,1.000000,86
114,47125,5353,192.168.0.1,224.0.0.251,01:00:5e:00:00:fb,0.000000,1.000000,86
73,53665,5353,192.168.0.1,224.0.0.251,01:00:5e:00:00:fb,0.000000,1.000000,86
106,58758,5353,192.168.0.1,224.0.0.251,01:00:5e:00:00:fb,0.000000,1.000000,86
45,53,49583,192.168.0.1,192.168.0.176,28:6d:97:77:cf:e8,0.008971,891.764743,956
31,53,41434,192.168.0.1,192.168.0.176,28:6d:97:77:cf:e8,0.057140,140.007310,956
67,53,59590,192.168.0.1,192.168.0.176,28:6d:97:77:cf:e8,0.009043,884.664294,988
46,34088,20002,192.168.0.1,192.168.0.255,ff:ff:ff:ff:ff:ff,0.000000,1.000000,368
98,42723,5353,192.168.0.1,224.0.0.251,01:00:5e:00:00:fb,0.000000,1.000000,86
83,35058,5353,192.168.0.1,224.0.0.251,01:00:5e:00:00:fb,0.000000,1.000000,86


In [63]:
le = preprocessing.LabelEncoder()
for column_name in x_train.columns:
    if x_train[column_name].dtype == object:
        x_train[column_name] = le.fit_transform(x_train[column_name])
    else:
        pass
for column_name in x_test.columns:
    if x_test[column_name].dtype == object:
        x_test[column_name] = le.fit_transform(x_test[column_name])
    else:
        pass
x_test

C:\Users\ishap\AppData\Local\Temp/ipykernel_13248/2613303932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[column_name] = le.fit_transform(x_train[column_name])
C:\Users\ishap\AppData\Local\Temp/ipykernel_13248/2613303932.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column_name] = le.fit_transform(x_test[column_name])


,source_port,dest_port,source_ip,dest_ip,dest_mac,flow_duration,flow_rate,flow_volume
48,40917,5353,1,3,0,0.000000,1.000000,86
114,47125,5353,1,3,0,0.000000,1.000000,86
73,53665,5353,1,3,0,0.000000,1.000000,86
106,58758,5353,1,3,0,0.000000,1.000000,86
45,53,49583,1,1,2,0.008971,891.764743,956
31,53,41434,1,1,2,0.057140,140.007310,956
67,53,59590,1,1,2,0.009043,884.664294,988
46,34088,20002,1,2,3,0.000000,1.000000,368
98,42723,5353,1,3,0,0.000000,1.000000,86
83,35058,5353,1,3,0,0.000000,1.000000,86


In [27]:
knn = KNeighborsClassifier(n_neighbors=30)
knn = Pipeline([('norm',StandardScaler()),('knn', knn)])
knn.fit(x_train,y_train)
pred_values = knn.predict(x_test)
print(list(set(pred_values)))
pred_dict = {}
for a in pred_values:
    if a not in pred_dict:
        pred_dict[a] = 1
    else:
        pred_dict[a] += 1
print (pred_dict)

['60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0']
{'60:32:b1:64:50:6d': 37, 'ee:3a:28:eb:91:a0': 13}


C:\Users\ishap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [32]:
conf = confusion_matrix(y_test,pred_values)
conf

array([[36,  0,  0],
       [ 1,  0,  0],
       [ 0,  0, 13]], dtype=int64)

In [33]:
#pd.crosstab(y_test, pred_values, rownames = ['Actual'], colnames =['Predicted'], margins = True)
print(classification_report(y_test, pred_values))

                   precision    recall  f1-score   support

60:32:b1:64:50:6d       0.97      1.00      0.99        36
b4:fb:e3:fb:bc:5d       0.00      0.00      0.00         1
ee:3a:28:eb:91:a0       1.00      1.00      1.00        13

         accuracy                           0.98        50
        macro avg       0.66      0.67      0.66        50
     weighted avg       0.96      0.98      0.97        50



C:\Users\ishap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ishap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ishap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
from sklearn import svm
clf = svm.SVC(kernel='linear') # Linear Kernel
y_train = y_train.values.flatten()

In [57]:
clf.fit(x_train, y_train)

SVC(kernel='linear')

In [58]:
pred_values = clf.predict(x_test)

In [59]:
pred_values

array(['60:32:b1:64:50:6d', '60:32:b1:64:50:6d', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0', 'ee:3a:28:eb:91:a0',
       'ee:3a:28:eb:91:a0', '60:32:b1:64:50:6d', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0', '60:32:b1:64:50:6d',
       'ee:3a:28:eb:91:a0', 'ee:3a:28:eb:91:a0', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', '60:32:b1:64:50:6d', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0', 'ee:3a:28:eb:91:a0',
       '60:32:b1:64:50:6d', '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0',
       '60:32:b1:64:50:6d', '60:32:b1:64:50:6d', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0', '60:32:b1:64:50:6d',
       '60:32:b1:64:50:6d', '60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0',
       '60:32:b1:64:50:6d', '60:32:b1:64:50:6d', '60:32:b1:64:50:6d',
       '60:32:b1:64:

In [60]:
print(list(set(pred_values)))
pred_dict = {}
for a in pred_values:
    if a not in pred_dict:
        pred_dict[a] = 1
    else:
        pred_dict[a] += 1
print (pred_dict)

['60:32:b1:64:50:6d', 'ee:3a:28:eb:91:a0']
{'60:32:b1:64:50:6d': 36, 'ee:3a:28:eb:91:a0': 14}


In [61]:
print(classification_report(y_test, pred_values))

                   precision    recall  f1-score   support

60:32:b1:64:50:6d       1.00      1.00      1.00        36
b4:fb:e3:fb:bc:5d       0.00      0.00      0.00         1
ee:3a:28:eb:91:a0       0.93      1.00      0.96        13

         accuracy                           0.98        50
        macro avg       0.64      0.67      0.65        50
     weighted avg       0.96      0.98      0.97        50



C:\Users\ishap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ishap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ishap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
